## Import libraries

In [ ]:
import tensorflow as tf
from tensorflow.python.data.ops.dataset_ops import PrefetchDataset
from tensorflow.python.framework.ops import Tensor
from tensorflow.python.keras import backend as K
from tensorflow.python.ops import array_ops
from tensorflow.python.ops import math_ops
from keras.callbacks import Callback

## Configs

In [ ]:
args = Namespace(
    train=True,
    test=False,
    inference=True,
    seed=42,
    folds=5,
    cv_method="stacking", # available methods: single, stacking, full
    data_path=Path("../input/ubiquant-market-prediction-half-precision-pickle"),
    tf_record_dataset_path=Path("../input/purged-5fold-tf-records"),
    model_path=Path("../input/5-fold-tf-models"),
    BATCH_SIZE = 4096
)

## Create TF dataset

In [ ]:
def decode_function(record_bytes):
      return tf.io.parse_single_example(
      # Data
      record_bytes,
      # Schema
      {
          "features": tf.io.FixedLenFeature([300], dtype=tf.float32),
          "time_id": tf.io.FixedLenFeature([], dtype=tf.int64),
          "year": tf.io.FixedLenFeature([], dtype=tf.int64),
          "quarter": tf.io.FixedLenFeature([], dtype=tf.int64),
          "cluster": tf.io.FixedLenFeature([num_clusters], dtype=tf.int64), 
          "sub_section": tf.io.FixedLenFeature([num_sub_sections], dtype=tf.int64),
          "main_section": tf.io.FixedLenFeature([num_main_sections], dtype=tf.int64),
          "market_value": tf.io.FixedLenFeature([], dtype=tf.int64),
          "time_ids_count": tf.io.FixedLenFeature([], dtype=tf.int64),
          "num_or_cat": tf.io.FixedLenFeature([], dtype=tf.int64),
          "target": tf.io.FixedLenFeature([], dtype=tf.float32)
      }
  )
    
def preprocess(item):
    return (item["features"], 
            item["time_id"],
            item["time_ids_count"], 
            item["num_or_cat"]
           ), item["target"]

def make_dataset(file_paths, batch_size=args.BATCH_SIZE, mode="train"):
    ds = tf.data.TFRecordDataset(file_paths)
    ds = ds.map(decode_function)
    ds = ds.map(preprocess)
    if mode == "train":
        ds = ds.shuffle(batch_size * 4)
    ds = ds.batch(batch_size).cache().prefetch(tf.data.AUTOTUNE)
    return ds

# Build model

## Utils

In [ ]:
def correlation(x, y, axis=-2):
    """Metric returning the Pearson correlation coefficient of two tensors over some axis, default -2."""
    x = tf.convert_to_tensor(x)
    y = math_ops.cast(y, x.dtype)
    n = tf.cast(tf.shape(x)[axis], x.dtype)
    xsum = tf.reduce_sum(x, axis=axis)
    ysum = tf.reduce_sum(y, axis=axis)
    xmean = xsum / n
    ymean = ysum / n
    
    xvar = tf.reduce_sum(tf.math.squared_difference(x, xmean), axis=axis)
    yvar = tf.reduce_sum(tf.math.squared_difference(y, ymean), axis=axis)

    cov = tf.reduce_sum((x - xmean) * (y - ymean), axis=axis)
    corr = cov / tf.sqrt(xvar * yvar)
    
    return tf.constant(1.0, dtype=x.dtype) - corr

def correlation_loss(x,y, axis=-2):
    """Loss function that maximizes the pearson correlation coefficient between the predicted values and the labels,
    while trying to have the same mean and variance"""
    x = tf.convert_to_tensor(x)
    y = math_ops.cast(y, x.dtype)
    n = tf.cast(tf.shape(x)[axis], x.dtype)
    xsum = tf.reduce_sum(x, axis=axis)
    ysum = tf.reduce_sum(y, axis=axis)
    xmean = xsum / n
    ymean = ysum / n
    xsqsum = tf.reduce_sum( tf.math.squared_difference(x, xmean), axis=axis)
    ysqsum = tf.reduce_sum( tf.math.squared_difference(y, ymean), axis=axis)
    cov = tf.reduce_sum( (x - xmean) * (y - ymean), axis=axis)
    corr = cov / tf.sqrt(xsqsum * ysqsum)
    return tf.convert_to_tensor( K.mean(tf.constant(1.0, dtype=x.dtype) - corr ) , dtype=tf.float32 )

def ccc(x, y):
    x_mean = tf.reduce_mean(x)
    y_mean = tf.reduce_mean(y)
    covariance = (x - x_mean) * (y - y_mean)
    x_var = tf.reduce_mean(tf.math.squared_difference(x, x_mean))
    y_var = tf.reduce_mean(tf.math.squared_difference(y, y_mean))
    return 2 * covariance / (x_var + y_var + tf.math.square(x_mean - y_mean) + 1e-7)

def ccc_loss(x, y):
    return 1.0 - ccc(x, y)

def scheduler(epoch, lr):
    print(f'LR is {lr}')
    return lr

class EarlyStoppingByLimits(Callback):
    def __init__(self, train_limit=0.5, val_limit=0.5):
        super(Callback, self).__init__()
        self.train_limit = train_limit
        self.val_limit = val_limit

    def on_epoch_end(self, epoch, logs={}):
        loss = logs.get('loss')
        val_loss = logs.get('val_loss')
        if loss < self.train_limit or val_loss < self.val_limit:
            self.model.stop_training = True

### DNN

In [ ]:
def get_model():
    # features
    features_inputs = tf.keras.Input((300, ), dtype=tf.float32)
    feature_x = tf.keras.layers.Dense(256, activation='swish')(features_inputs)
    feature_x = tf.keras.layers.Dense(256, activation='swish')(feature_x)
    feature_x = tf.keras.layers.Dense(256, activation='swish')(feature_x)
    
    # time_id
    time_id_inputs = tf.keras.Input((1, ), dtype=tf.int64)
    time_id_x = tf.keras.layers.Dense(16, activation='swish')(time_id_inputs)
    time_id_x = tf.keras.layers.Dense(16, activation='swish')(time_id_x)
    time_id_x = tf.keras.layers.Dense(16, activation='swish')(time_id_x)
    
    # time_ids_count
    time_ids_count_inputs = tf.keras.Input((1, ), dtype=tf.int64)
    time_ids_count_x = tf.keras.layers.Dense(16, activation='swish')(time_ids_count_inputs)
    time_ids_count_x = tf.keras.layers.Dense(16, activation='swish')(time_ids_count_x)
    time_ids_count_x = tf.keras.layers.Dense(16, activation='swish')(time_ids_count_x)
    
    # num_or_cat
    num_or_cat_inputs = tf.keras.Input((1, ), dtype=tf.float32)

    x = tf.keras.layers.Concatenate(axis=1)([time_id_x, time_ids_count_x, feature_x, num_or_cat_inputs])
    
    x = tf.keras.layers.Dense(512, activation='swish', kernel_regularizer="l2")(x)
    x = tf.keras.layers.Dropout(0.1)(x)
    
    x = tf.keras.layers.Dense(128, activation='swish', kernel_regularizer="l2")(x)
    x = tf.keras.layers.Dropout(0.1)(x)
    
    x = tf.keras.layers.Dense(32, activation='swish', kernel_regularizer="l2")(x)
    x = tf.keras.layers.Dropout(0.1)(x)
    
    output = tf.keras.layers.Dense(1)(x)
    rmse = tf.keras.metrics.RootMeanSquaredError(name="rmse")
    
    model = tf.keras.Model(inputs=[features_inputs, time_id_inputs, time_ids_count_inputs, num_or_cat_inputs], outputs=[output])
#     model.compile(optimizer=tf.optimizers.Adam(1e-3), loss='mse', metrics=['mse', "mae", "mape", rmse, correlation])
    model.compile(optimizer=tf.optimizers.Adam(1e-4),  loss=ccc_loss, metrics=['mse', "mae", "mape", rmse, correlation])
    return model

## Show model

In [ ]:
model = get_model()
model.summary()
tf.keras.utils.plot_model(model, show_shapes=True)

# Train
## 5-fold

In [ ]:
%%time
models = []
tf.random.set_seed(args.seed)

for i in range(args.folds):
    train_path = args.tf_record_dataset_path.joinpath(f"fold_{i}_train.tfrecords")
    valid_path = args.tf_record_dataset_path.joinpath(f"fold_{i}_test.tfrecords")
    valid_ds = make_dataset([valid_path], mode="valid")
    model = get_model2()
    if args.train :
        train_ds = make_dataset([train_path])
        sched_callback = tf.keras.callbacks.LearningRateScheduler(scheduler)
        checkpoint = tf.keras.callbacks.ModelCheckpoint(f"model_{i}.tf", monitor="val_correlation", mode="min", save_best_only=True, save_weights_only=True)
        early_stop = tf.keras.callbacks.EarlyStopping(patience=10)
#         early_stop = EarlyStoppingByLimits (train_limit=0.75, val_limit=0.85)
        history = model.fit(train_ds, epochs=30, validation_data=valid_ds, shuffle=True, callbacks=[sched_callback, checkpoint, early_stop])
        for metric in ["loss", "mae", "mape", "rmse", "correlation"]:
            pd.DataFrame(history.history, columns=[metric, f"val_{metric}"]).plot()
            plt.title(metric.upper())
            plt.show()
        del train_ds
    else:
        model.load_weights(args.model_path.joinpath(f"model_{i}.tf"))
    models.append(model)
    y_vals = []
    for _, y in valid_ds:
        y_vals += list(y.numpy().reshape(-1))
    y_val = np.array(y_vals)
    pearson_score = stats.pearsonr(model.predict(valid_ds).reshape(-1), y_val)[0]
    print(f"Pearson Score: {pearson_score}")

    del valid_ds
    gc.collect()

# Inference with PCA averaging

## Utils

In [ ]:
pca = PCA(n_components=1)

def preprocess_test(feature, 
                    time_id, 
                    time_ids_count, 
                    num_or_cat, 
                    ):
    return (feature, 
            time_id, 
            time_ids_count, 
            num_or_cat,
           ), 0

def make_test_dataset(feature, 
                      time_id, 
                      time_ids_count, 
                      num_or_cat,
                      batch_size=args.BATCH_SIZE):
    ds = tf.data.Dataset.from_tensor_slices(((feature, 
                                              time_id, 
                                              time_ids_count, 
                                              num_or_cat
                                             )))
    ds = ds.map(preprocess_test)
    ds = ds.batch(batch_size).cache().prefetch(tf.data.experimental.AUTOTUNE)
    return ds

def pca_averaging(models, ds):
    y_preds = []
    for model in models:
        y_pred = model.predict(ds)
        y_preds.append(y_pred)
    res = np.hstack(y_preds)
    if len(res) > 1:
        res = -1 * pca.fit_transform(res)
    else:
        res = np.mean(res, axis=1)
    return res

## Make prediction with stacking

In [ ]:
env = ubiquant.make_env()
iter_test = env.iter_test() 

In [ ]:
time_id = 1220
test_time_id_len = 942961

if args.inference:
    features = [f"f_{i}" for i in range(300)]
    
    for (test_df, sample_prediction_df) in iter_test:
        # extract time_id
        try:
            test_time_id = int(test_df['row_id'].values[0].split('_')[0]) # extract time_id from row_id
            test_df["time_id"] = test_time_id
        except:
            test_df["time_id"] = time_id
            test_time_id = 0
        # in case of error just increase time_id on 1
        if test_time_id:
            time_id = test_time_id + 1
        else:
            test_df["time_id"] = time_id
            time_id += 1
        
        # get number of time_ids in the test_df
        test_df['time_ids_count'] = len(test_df)
        
        # create num_or_cat feature for the test_df
        unqiues = len(test_df['f_7'].unique())
        if unqiues/len(test_df) > 0.5:
            test_df['num_or_cat'] = 0
        else:
            test_df['num_or_cat'] = 1
        
        ds = make_test_dataset(test_df[features], 
                                test_df["time_id"],
                                test_df["time_ids_count"], 
                                test_df["num_or_cat"])
        
        sample_prediction_df['target'] = pca_averaging(models, ds)
        env.predict(sample_prediction_df)